# **The bigARTM part is [here](https://github.com/deniskapel/Compling_course/blob/master/KD_hw_6_part2.ipynb)**

# Gensim

## Preprocess texts

In [ ]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
morph = MorphAnalyzer()

In [ ]:
stops = set(stopwords.words('russian'))

Normalizing texts

In [ ]:
class TextNormalizer():
    
    # Class for fast text-normalization
        
    def opt_normalize(self, texts, top=None) -> list:
        
        texts = self.tokenize_texts(texts)
        
        uniq = Counter()
        [uniq.update(text) for text in texts]

        norm_uniq = { # build cache
            word:morph.parse(
                word
            )[0].normal_form for word, _ in uniq.most_common(top)}

        norm_texts = self.normalize_texts(texts, norm_uniq)
        
        return norm_texts
    
    def tokenize_texts(self, texts):
        """ a text tokenizer for a list of texts """
        return ([self.tokenize(text.lower()) for text in texts])


    def tokenize(self, text) -> list:
        """ a single text tokenizer """
        punct = string.punctuation + '«»—…–“”#'
        words = [word.strip(punct) for word in text.split()]
        words = [word for word in words if word]

        return words
    
    def normalize_texts(self, texts, norm_uniq) -> list:
        """ returns lemmas and removes stopwords and punctuation """
        norm_texts = []
        for text in texts:
            # lemmas from cashe
            norm_words = [norm_uniq.get(word) for word in text]
            #remove punctuation
            
            norm_words = [word for word in norm_words if (
                word) and (word not in stops)]
            norm_texts.append(norm_words)

        return norm_texts

In [ ]:
texts = open('data/wiki_data.txt').read().splitlines()[:10000]

In [ ]:
%%time
texts = TextNormalizer().opt_normalize(texts, 30000)

CPU times: user 2.63 s, sys: 58.9 ms, total: 2.69 s
Wall time: 2.69 s


In [ ]:
texts[0][:5]

['нижегородский', 'сельский', 'посёлок', 'район', 'нижегородский']

build ngrams

In [ ]:
# play around with threshold
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.45)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

# Build dictionary for models

In [ ]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.filter_extremes(no_above=0.2, no_below=15)
dictionary.compactify()
print(dictionary)

Dictionary(6846 unique tokens: ['1', '1,2', '1,5', '12', '14']...)


transform to corpora

### No TfIdf

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
lda1 = gensim.models.LdaMulticore(corpus,
                                 100,
                                 id2word=dictionary,
                                 passes=7)

lda1.save('data/models/lda1/lda1.model')

In [ ]:
lda2 = gensim.models.LdaMulticore(corpus,
                                 50,
                                 id2word=dictionary,
                                 passes=10)

lda2.save('data/models/lda2/lda.model')

In [ ]:
lda3 = gensim.models.LdaMulticore(corpus,
                                 150,
                                 id2word=dictionary,
                                 passes=5)

lda3.save('data/models/lda3/lda.model')

### With TfIdf

In [ ]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
tfidf_corpus = tfidf[corpus]

In [ ]:
tfidf_lda1 = gensim.models.LdaMulticore(tfidf_corpus,
                                 100,
                                 id2word=dictionary,
                                 passes=7)

tfidf_lda1.save('data/models/tfidf_lda1/tfidf_lda.model')

In [ ]:
tfidf_lda2 = gensim.models.LdaMulticore(tfidf_corpus,
                                 50,
                                 id2word=dictionary,
                                 passes=10)

tfidf_lda2.save('data/models/tfidf_lda2/tfidf_lda.model')

In [ ]:
tfidf_lda3 = gensim.models.LdaMulticore(tfidf_corpus,
                                 150,
                                 id2word=dictionary,
                                 passes=5)

tfidf_lda3.save('data/models/tfidf_lda3/tfidf_lda.model')

### compare models

In [ ]:
# load models
model1 = gensim.models.LdaModel.load(
    'data/models/lda1/lda1.model')
model2 = gensim.models.LdaModel.load(
    'data/models/lda2/lda.model')
model3 = gensim.models.LdaModel.load(
    'data/models/lda3/lda.model')
tfidf1 = gensim.models.LdaModel.load(
    'data/models/tfidf_lda1/tfidf_lda.model')
tfidf2 = gensim.models.LdaModel.load(
    'data/models/tfidf_lda2/tfidf_lda.model')
tfidf3 = gensim.models.LdaModel.load(
    'data/models/tfidf_lda3/tfidf_lda.model')

In [ ]:
def get_topics(model):
    topics = []
    for id, topic in model.show_topics(num_topics=30, formatted=False):
        topic = [word for word, _ in topic]
        topic.insert(0, str(id))
        topics.append(topic)
    
    return topics

In [ ]:
topics1 = get_topics(model1)
topics2 = get_topics(model2)
topics3 = get_topics(model3)
topics4 = get_topics(tfidf1)
topics5 = get_topics(tfidf2)
topics6 = get_topics(tfidf3)

In [ ]:
for topic in topics1: # regular lda 100x7
    print(" ".join(topic))

73 памятник ростовский хутор культура история объект церковь культурный n наследие
93 упражнение ян филиппина серебряный ботсвана джованни рой даваться венеция ребёнок
8 чёрный флаг белый пешка король поле ход красный цвет сторона
74 строительство проект новый город рубль работа мост здание проведение 2010
50 марка почтовый финляндия почта выпуск марк финский остров надпись рисунок
28 церковь епархия епископ рог 1-й 1-2-й участник команда lotus век
65 житомирский украина 2001 перепись код площадь почтовый р-н индекс телефонный
25 звезда бразилия b собака обсерватория срок планета 2009 геологический порода
39 компания версия система программа игра разработка windows мочь поддержка использовать
66 война торговля союз договор город гитлер купец торговый товар москва
3 клуб храм икона футбольный церковь стадион сезон кубок город дивизион
78 паровоз адам польша вместе павел дорога вальтер немецкий война март
15 город население центр состав деревня житель территория расположить век название


In [ ]:
for topic in topics2: # regular lda 50x10
    print(" ".join(topic))

31 фильм сын роль граф брат смерть дочь i отец её
26 г н м п русский газета москва александр л кладбище
40 клуб матч команда сезон сборная состав лига провести чемпионат забить
46 автомобиль модель подвеска система иметь двигатель новый колесо задний версия
21 здание дом построить вода проект строительство завод станция новый город
28 мочь являться система иметь использовать весь использоваться вид число например
44 турнир чемпионат мир сезон команда победа выиграть матч проиграть финал
16 суд закон право дело гражданин латвия решение судебный сельсовет верховный
43 город век герб часть слово цвет язык название племя белый
48 россия улица москва день предприятие площадь украина город сергей дом
45 флаг цвет социальный полотнище красный символ российский ширина полоса обеспечение
32 пешка король окончание ферзь позиция блюдо сторона мясо поле белый
23 река библиотека бассейн фестиваль расположить приток начало впадать число территория
24 город войско часть армия река противник км бой ди

In [ ]:
for topic in topics3: # regular lda 150x5
    print(" ".join(topic))

145 храм церковь собор монастырь святой век епископ здание православный русский
46 вино военный сша компания частный являться мочь оон иметь сорт
10 медведь ричард её мультсериал племя являться использовать вещество страна ребёнок
67 князь иван робот московский великий иванович век москва iii русский
139 город задача проект 1 являться 2 весь всё число коммуна
18 автомобиль задний модель двигатель подвеска кузов передний колесо иметь мост
29 сельский состав хутор центр ростовский поселение входить посёлок река станица
89 российский федерация театр авиакомпания россия общественный 2010 компания валерий являться
106 ссср день сеть войско пво оборона транспортный праздник анализ воскресение
34 г ян сын российский министр дело дочь институт дмитриевич софья
37 население город станица казак округ семья перепись житель средний сша
147 вагон день иметь весь адам мочь слово звезда праздник всё
142 медаль выиграть количество норвегия 380 игра оркестр золотой епархия канада
99 военный польша войн

In [ ]:
model1.log_perplexity(corpus[:1000]), model2.log_perplexity(corpus[:1000]), model3.log_perplexity(corpus[:1000])

(-12.529603011976924, -9.919772060404574, -16.854604284921272)

In [ ]:
coherence_model_lda1 = gensim.models.CoherenceModel(model=model1,
                                                    texts=texts,
                                                    dictionary=dictionary,
                                                    coherence='c_v')

coherence_model_lda2 = gensim.models.CoherenceModel(model=model2,
                                                    texts=texts,
                                                    dictionary=dictionary,
                                                    coherence='c_v')

coherence_model_lda3 = gensim.models.CoherenceModel(model=model3,
                                                    texts=texts,
                                                    dictionary=dictionary,
                                                    coherence='c_v')

coherence_model_lda1.get_coherence(), coherence_model_lda2.get_coherence(), coherence_model_lda3.get_coherence()

(0.49964053048061435, 0.5593595436815606, 0.4770401638735298)

### Comparing models without tfidf
After analysing samples of 30 topics, it is possible to say that model with 50x10 paramenters performs relatively better. Results of log_perplexity and coherence can support it.

**Best topics**:

40 клуб матч команда сезон сборная состав лига провести чемпионат забить

8 музей искусство художник век работа выставка произведение картина художественный современный

30 состав территория центр волость входить пункт станица население войти часть

In [ ]:
for topic in topics4: # tfidf lda 100x7
    print(" ".join(topic))

46 инвалид сердце 260 аргентинский актёрский лечение госпиталь кевин ангел реабилитация
92 олимпийский игра летний зимний медаль завоевать принимать участие история сборная
96 святой жена муж имя актриса дочь трое сын её воля
99 женский химический али скульптор белый художница нии скульптура иркутский античность
22 ленин боливия 19 варшавский 79 боковой 1969 партизан проектирование 1917
72 вещество арена магнус тайвань король сигурд инга жидкость мозг волокно
24 монголия сингл хорватия органический грэмми статистика hot billboard испанский рудник
15 самолёт эфир истребитель приём полёт ввс новость рота ствол general
3 каменский области.############ть комплект ямайка камера плато ткань популяция войти world
40 корея мюнхен художник 1972 мексика живопись музей выставка картина женщина
84 поезд бюро никольский раствор соборный de автоматический тонна скорость км/ча
57 николай месторождение математик тёмный уральский минерал водиться dark монах цска
85 коростенский 4142 остров ладожский ар

In [ ]:
for topic in topics5: # tfidf lda 50x10
    print(" ".join(topic))

35 раунд спортсмен дистанция квалификация муравей занятой независимо утешительный сидней термин
31 ворота язык иванович социалистический труд армянский литовский герой индия преподавать
5 церковь святой храм собор икона монастырь епархия православный католический дева
15 граф король сын i ii князь графство де iii герцог
9 албания балка верфь 1906 фиджи лодка подводный нью-джерси вилла ag
8 фильм война г советский армия школа военный работать роль получить
42 квебек турок юрий пехотный николаевский севастополь георгий мастерс петрович 65
39 хутор ростовский сельский области.############входить река посёлок станица верхнедонский км состав
20 значение сенегал аббревиатура человек.############являться рим 1960 впадение озеро боксёр гандбол
34 км расстояние м/ж сельский река район,######харьковский расположить харьковский течение берег
46 носитель монако литература тепловоз малайзия повесть писатель мощность intel сан-франциско
22 зачёт прыжок фёдор космический ракета ван вкп(б арестовать ц

In [ ]:
for topic in topics6: # tfidf lda 150x5
    print(" ".join(topic))

81 встречаться шестой муж её жена брать жить мать дочь джордж
90 турнир смотреть чемпионат мир финал раунд выиграть чемпион проиграть вещество
16 китай китайский пекин чжан династия женщина тан архитектура традиционный история
111 крест пьеса постановка поставить сюжетный театр сцена драматург визуальный героиня
126 университет газета книга профессор факультет русский институт н нью-йорк литература
122 святой румыния румынский житие епископ племянник родство 1989 имя вероятно
4 макс электрический убить тюрьма молния питер доктор зелёный пытаться мочь
98 философия руда педагогический колумбийский колледж нью-джерси премия николас организм г
40 майк песня написать хит the альбом records группа записать is
28 иванович николай н оркестр данные рабочий анализ инструмент возможность продукт
56 овручский украина лист монастырь икона насекомое цветок упоминаться венгрия монах
148 оркестр инструмент змея вселенная музыка мнение рассказ планета автор крестьянин
135 хутор ростовский области.#####

In [ ]:
tfidf1.log_perplexity(corpus[:1000]), tfidf2.log_perplexity(corpus[:1000]), tfidf3.log_perplexity(corpus[:1000])

(-11.262671233880246, -8.946754967565905, -28.05496668806068)

In [ ]:
coherence_tfidf_lda1 = gensim.models.CoherenceModel(model=tfidf1,
                                                    texts=texts,
                                                    dictionary=dictionary,
                                                    coherence='c_v')

coherence_tfidf_lda2 = gensim.models.CoherenceModel(model=tfidf2,
                                                    texts=texts,
                                                    dictionary=dictionary,
                                                    coherence='c_v')

coherence_tfidf_lda3 = gensim.models.CoherenceModel(model=tfidf3,
                                                    texts=texts,
                                                    dictionary=dictionary,
                                                    coherence='c_v')

coherence_tfidf_lda1.get_coherence(), coherence_tfidf_lda2.get_coherence(), coherence_tfidf_lda3.get_coherence()

(0.49866245961659855, 0.5574097093390988, 0.4119803382564534)

### Comparing models with tfidf

After analysing samples of 30 topics, it is possible to say that again the model with 50x10 paramenters performs relatively better. It seems the number of passes has a higher impact on quality than number of topics. Results of log_perplexity and coherence can support it.


**Best topics**:

5 церковь святой храм собор икона монастырь епархия православный католический дева

27 клуб чемпионат матч команда сезон турнир кубок мир чемпион игрок

47 альбом песня the студийный певица in and розыгрыш сингл выпустить

## Comparing best models

The model without tfidf seems to produce better shaped topics, but its metrics are slightly worse. Overall, well-shaped topics intersect, e.g: sports, geographic landmarks, religion.

For now, it seems that a model with higher number of topics and passes (like the one we saw in class) performs better.

#  Sklearn

In [ ]:
from sklearn.decomposition import NMF
from joblib import dump, load
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

preprocess

In [ ]:
stexts = [' '.join(text) for text in texts]

count_vec = CountVectorizer(max_features=3000,
                             min_df=8,
                             max_df=0.2,
                             ngram_range=(1,2))

X_count = count_vec.fit_transform(stexts)

In [ ]:
tfidf_vec = TfidfVectorizer(max_features=2000,
                             min_df=10,
                             max_df=0.1,
                             ngram_range=(1,2))

X_tfidf = tfidf_vec.fit_transform(stexts)

CountVectorizer models

In [ ]:
skl1 = NMF(n_components=100)
skl1.fit(X_count)
dump(skl1, 'data/models/skl1/model.joblib')

['data/models/skl1/model.joblib']

In [ ]:
skl2 = NMF(n_components=50)
skl2.fit(X_count)
dump(skl2, 'data/models/skl2/model.joblib')

['data/models/skl2/model.joblib']

In [ ]:
skl3 = NMF(n_components=150)
skl3.fit(X_count)
dump(skl3, 'data/models/skl3/model.joblib')

['data/models/skl3/model.joblib']

TfIdfVectorizer moderls

In [ ]:
tfidf_skl1 = NMF(n_components=100)
tfidf_skl1.fit(X_count)
dump(tfidf_skl1, 'data/models/tfidf_skl1/model.joblib')

['data/models/tfidf_skl1/model.joblib']

In [ ]:
tfidf_skl2 = NMF(n_components=50)
tfidf_skl2.fit(X_count)
dump(tfidf_skl2, 'data/models/tfidf_skl2/model.joblib')

['data/models/tfidf_skl2/model.joblib']

In [ ]:
tfidf_skl3 = NMF(n_components=150)
tfidf_skl3.fit(X_count)
dump(tfidf_skl3, 'data/models/tfidf_skl3/model.joblib')

['data/models/tfidf_skl3/model.joblib']

**use count_vec models**

In [ ]:
skl1 = load('data/models/skl1/model.joblib')
skl2 = load('data/models/skl2/model.joblib')
skl3 = load('data/models/skl3/model.joblib')

In [ ]:
cfeat_names = count_vec.get_feature_names()
tfeat_names = tfidf_vec.get_feature_names()

In [ ]:
top_words1 = skl1.components_.argsort()[:30,:-9:-1] # number of components = 100

for i in range(top_words1.shape[0]):
    words = [cfeat_names[j] for j in top_words1[i]]
    print(i, "--".join(words))

0 подвеска--автомобиль--колесо--рычаг--поперечный--поворот--задний--ось
1 войско--фронт--армия--город--январь--противник--бой--овладеть
2 новый--версия--10--выпуск--выпустить--включать--апрель--проект
3 игра--of--2010--2010 год--black--карта--2011--студия
4 турнир--разряд--финал--пара--рейтинг--открытый--открытый чемпионат--парный
5 нансен--экспедиция--фрам--лёд--весь--команда--северный--свердруп
6 линь--лян--китайский--её--архитектура--китай--пекин--дом
7 генерал--армия--корпус--конный--февраль--войско--командование--сентябрь
8 уезд--округ--специальный--городской--специальный район--состав--год уезд--городской уезд
9 москва--купец--россия--торговля--век--русский--товар--новгород
10 житомирский--код--житомирский область--2001--2001 год--перепись--украина--год составлять
11 брак--право--однополый--россия--российский--мочь--суд--семейный
12 турнир--раунд--выйти--чемпионат--финал--открытый--смочь--открытый чемпионат
13 религиозный--передача--здание--организация--имущество--церковь--2010--

In [ ]:
top_words2 = skl2.components_.argsort()[:30,:-9:-1] # number of components = 50

for i in range(top_words2.shape[0]):
    words = [cfeat_names[j] for j in top_words2[i]]
    print(i, "--".join(words))

0 подвеска--автомобиль--колесо--рычаг--поперечный--поворот--задний--ось
1 войско--армия--фронт--город--январь--противник--бой--овладеть
2 вид--часть--являться--иметь--век--растение--цвет--мочь
3 игра--of--2010--2010 год--black--карта--2011--студия
4 турнир--мир--чемпионат--финал--сезон--открытый--открытый чемпионат--разряд
5 нансен--экспедиция--фрам--лёд--весь--команда--северный--свердруп
6 линь--лян--её--китайский--архитектура--китай--дом--пекин
7 генерал--армия--штаб--август--военный--корпус--сентябрь--начальник
8 уезд--округ--городской--специальный--специальный район--состав--год уезд--городской уезд
9 экономический--общество--россия--вольный--российский--развитие--деятельность--труд
10 житомирский--код--украина--житомирский область--площадь--2001--2001 год--перепись
11 брак--право--однополый--россия--мочь--российский--суд--семейный
12 раунд--проиграть--счёт--турнир--первый раунд--второй--квалификация--открытый
13 церковь--храм--монастырь--святой--собор--епископ--русский--православн

In [ ]:
top_words3 = skl3.components_.argsort()[:30,:-9:-1] # number of components = 150

for i in range(top_words3.shape[0]):
    words = [cfeat_names[j] for j in top_words3[i]]
    print(i, "--".join(words))

0 подвеска--автомобиль--колесо--рычаг--поперечный--поворот--задний--ось
1 войско--фронт--армия--город--январь--противник--бой--овладеть
2 октябрь--июнь--май--декабрь--ноябрь--сентябрь--апрель--январь
3 игра--of--2010--2010 год--black--карта--2011--студия
4 турнир--финал--разряд--пара--рейтинг--открытый--открытый чемпионат--парный
5 весь--всё--ещё--большой--самый--должный--несколько--практически
6 линь--лян--китайский--её--архитектура--китай--пекин--дом
7 состав--округ--свердловский--муниципальный--образование--следующий--территория--муниципальный образование
8 уезд--округ--специальный--специальный район--городской--год уезд--состав--городской уезд
9 россия--российский--москва--русский--московский--федерация--российский федерация--2015
10 житомирский--код--житомирский область--2001--2001 год--перепись--год составлять--перепись 2001
11 брак--однополый--право--мочь--семейный--пара--семья--суд
12 турнир--раунд--выйти--чемпионат--финал--открытый--открытый чемпионат--смочь
13 религиозный--пе

### Comparing CountVec models

The higher number of components allows to shape topics better:

n=50: 11 брак--право--однополый--россия--мочь--российский--суд--семейный

n=100: 11 брак--право--однополый--россия--российский--мочь--суд--семейный

n=150: 11 брак--однополый--право--мочь--семейный--пара--семья--суд

**Best-shaped topics**

0 подвеска--автомобиль--колесо--рычаг--поперечный--поворот--задний--ось

16 батальон--полка--полк--противник--позиция--бой--дивизия--атака

28 нансен--экспедиция--фрам--лёд--команда--северный--свердруп--полюс

**use tfidf models**

In [ ]:
tfidf_skl1 = load('data/models/tfidf_skl1/model.joblib')
tfidf_skl2 = load('data/models/tfidf_skl2/model.joblib')
tfidf_skl3 = load('data/models/tfidf_skl3/model.joblib')

In [ ]:
top_words4 = tfidf_skl1.components_.argsort()[:30,:-9:-1] # number of components = 100

for i in range(top_words4.shape[0]):
    words = [cfeat_names[j] for j in top_words4[i]]
    print(i, "--".join(words))

0 подвеска--автомобиль--колесо--рычаг--поперечный--поворот--задний--ось
1 войско--фронт--армия--город--январь--противник--бой--овладеть
2 новый--всё--весь--тоннель--вагон--однако--получить--название
3 игра--of--2010--2010 год--black--карта--2011--студия
4 турнир--финал--чемпионат--открытый--открытый чемпионат--разряд--мир--пара
5 нансен--экспедиция--фрам--лёд--весь--команда--северный--свердруп
6 линь--лян--её--китайский--архитектура--китай--пекин--дом
7 состав--округ--свердловский--образование--муниципальный--следующий--территория--образовать
8 уезд--округ--специальный--городской--специальный район--год уезд--состав--городской уезд
9 государственный--российский--ссср--совет--школа--федерация--министерство--председатель
10 житомирский--код--житомирский область--2001--украина--2001 год--перепись--год составлять
11 брак--однополый--право--россия--российский--мочь--суд--семейный
12 бой--раунд--удар--победа--боец--однако--провести--первый раунд
13 религиозный--передача--здание--организация-

In [ ]:
top_words5 = tfidf_skl2.components_.argsort()[:30,:-9:-1] # number of components = 50

for i in range(top_words5.shape[0]):
    words = [cfeat_names[j] for j in top_words5[i]]
    print(i, "--".join(words))

0 подвеска--автомобиль--колесо--рычаг--поперечный--поворот--задний--ось
1 войско--армия--фронт--город--январь--противник--бой--овладеть
2 вид--мочь--животное--являться--иметь--некоторый--язык--часть
3 игра--of--2010--2010 год--black--карта--2011--студия
4 турнир--финал--чемпионат--открытый--открытый чемпионат--разряд--мир--пара
5 нансен--экспедиция--фрам--полюс--команда--весь--лёд--северный
6 линь--лян--её--китайский--архитектура--китай--дом--пекин
7 генерал--армия--штаб--август--корпус--сентябрь--военный--начальник
8 уезд--округ--городской--специальный--специальный район--состав--год уезд--городской уезд
9 ссср--советский--член--союз--совет--работать--государственный--школа
10 житомирский--код--украина--житомирский область--2001--2001 год--перепись--год составлять
11 брак--право--однополый--россия--мочь--российский--суд--семейный
12 раунд--проиграть--счёт--первый раунд--турнир--второй--квалификация--чемпионат
13 церковь--храм--монастырь--святой--собор--епископ--век--русский
14 корабль

In [ ]:
top_words6 = tfidf_skl3.components_.argsort()[:30,:-9:-1] # number of components = 150

for i in range(top_words6.shape[0]):
    words = [cfeat_names[j] for j in top_words6[i]]
    print(i, "--".join(words))

0 подвеска--автомобиль--колесо--рычаг--поперечный--поворот--задний--ось
1 войско--фронт--армия--город--январь--противник--бой--овладеть
2 новый--тоннель--название--однако--январь--больший--большой--старый
3 игра--of--2010--2010 год--black--карта--2011--студия
4 турнир--разряд--финал--пара--рейтинг--открытый--открытый чемпионат--парный
5 всё--весь--мочь--должный--друг--жизнь--бог--последний
6 линь--лян--китайский--её--архитектура--китай--пекин--дом
7 состав--округ--свердловский--муниципальный--образование--территория--следующий--муниципальный образование
8 уезд--округ--специальный--специальный район--городской--состав--год уезд--городской уезд
9 россия--российский--русский--московский--москва--история--2015--участие
10 житомирский--код--житомирский область--2001--2001 год--перепись--год составлять--перепись 2001
11 брак--однополый--право--мочь--россия--семейный--пара--российский
12 турнир--раунд--выйти--чемпионат--открытый--открытый чемпионат--финал--смочь
13 религиозный--передача--здан

### Comparing TfIdfVec models

The higher number of components allows to shape even subtopics:

4 турнир--разряд--финал--пара--рейтинг--открытый--открытый чемпионат--парный

12 турнир--раунд--выйти--чемпионат--открытый--открытый чемпионат--финал--смочь

22 олимпийский--олимпийский игра--игра--летний--летний олимпийский--участие--медаль--принимать

**Overall difference between sklearn tfidf and count models seems to be insignificant.**

Generally, it looks like sklearn works faster and shapes more specific topics better than gensim. 

Topics likes religion, army, sports appear in all models and with all parameters